In [1]:
%matplotlib inline
import mlconfig
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import psycopg2
from psycopg2 import sql
from IPython.display import display
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LinearRegression

In [2]:
conn = psycopg2.connect(user = mlconfig.user,
                              password = mlconfig.password,
                              host = mlconfig.hostname,
                              port = mlconfig.port,
                              database = mlconfig.database)

cur = conn.cursor()

# Print PostgreSQL version
cur.execute("SELECT * from profiles;")
record = cur.fetchall()

In [3]:
cur.execute("Select * FROM profiles LIMIT 0")
colnames = [desc[0] for desc in cur.description]
print(colnames)
p = ['username', 'agerange', 'gender', 'orientation', 'currentstatus', 'togethertime', 'relationshipdescription', 'selfeducation', 'selffinancial', 'selfconfidence', 'selfreligious', 'selfmaterialism', 'selfimage', 'selfoccupation', 'selfworkethic', 'selfhousehold', 'selfcommunication', 'selfartsy', 'selfcharitable', 'selfpurpose', 'selfstatus', 'selfcultured', 'selfselfcare', 'selfhonesty', 'selffamily']
print(len(p))

['username', 'agerange', 'gender', 'orientation', 'currentstatus', 'togethertime', 'relationshipdescription', 'selfeducation', 'selffinancial', 'selfconfidence', 'selfreligious', 'selfmaterialism', 'selfimage', 'selfoccupation', 'selfworkethic', 'selfhousehold', 'selfcommunication', 'selfartsy', 'selfcharitable', 'selfpurpose', 'selfstatus', 'selfcultured', 'selfselfcare', 'selfhonesty', 'selffamily', 'value1', 'selfvalue1', 'partnervalue1', 'value2', 'selfvalue2', 'partnervalue2', 'value3', 'selfvalue3', 'partnervalue3', 'value4', 'selfvalue4', 'partnervalue4', 'value5', 'selfvalue5', 'partnervalue5', 'selfappearance', 'selfsocial', 'selfshy', 'selfalpha', 'selfhumorous', 'selfspontaneous', 'selfgenerous', 'selfdriven', 'selfintuitive', 'selfsexual', 'selfopenminded', 'selfvibe', 'selfovergiving', 'selfdominant', 'type1', 'partnertype1', 'type2', 'partnertype2', 'type3', 'partnertype3', 'type4', 'partnertype4', 'type5', 'partnertype5', 's_decisionmakingprocess', 'qp_emotionalintellige

In [4]:

lovedata = pd.DataFrame(record)

lovedata = lovedata.dropna(subset=lovedata.columns[25:39])

In [5]:
# update list if there are any fields added to data
features = pd.DataFrame(columns=['Education/Knowledge/Street Smarts',
          'partner_education_score',
          'difference_education_score',
          'Financial Choices',
          'partner_financial',
          'difference_financial',
          'Confidence/Self-Esteem',
          'partner_confidence',
          'difference_confidence',
          'Religious/Spiritual Values',
          'partner_religious',
          'difference_religious',
          'Materialism/Superficiality',
          'partner_materialism',
          'difference_materialism',
          'Image/Fashion Sense/Body Modification',
          'partner_image',
          'differnece_image',
          'Occupation/Work Ethic/Self-Discipline',
          'partner_occupation',
          'difference_occupation',
          'Household Care, Maintenance and Cleanliness',
          'partner_house','difference_house',
          'Communication Style/Manners',
          'partner_communication',
          'difference_communication',
          'Artsy/Creative/Musical',
          'partner_artsy',
          'difference_artsy',
          'Charitable/Philanthropic',
          'partner_charity',
          'difference_charity',
          'Pursuing a Greater Purpose',
          'partner_purpose',
          'difference_purpose',
          'Social Status/Sociability',
          'partner_social',
          'difference_social',
          'Cultured/Well-traveled/"Woke"',
          'partner_culture',
          'difference_culture',
          'Self-Care/Personal Hygiene/Cleanliness',
          'partner_care','difference_care',
          'Honesty/Dependable/Reliable',
          'partner_honesty','difference_honesty','Family Values','partner_family','difference_family'])
features

,Education/Knowledge/Street Smarts,partner_education_score,difference_education_score,Financial Choices,partner_financial,difference_financial,Confidence/Self-Esteem,partner_confidence,difference_confidence,Religious/Spiritual Values,...,difference_culture,Self-Care/Personal Hygiene/Cleanliness,partner_care,difference_care,Honesty/Dependable/Reliable,partner_honesty,difference_honesty,Family Values,partner_family,difference_family


In [6]:
# iterate thru dataframe and use columns from 
for i, j in lovedata.iterrows():
    if i == 0:
        continue
        
    ls = [0 for x in range(len(features.columns))]
        
    for k in range(25,39,3):
        if j[k] in features.columns:
            ls[features.columns.get_loc(j[k])] = int(j[k+1])
            ls[features.columns.get_loc(j[k]) + 1] = int(j[k+2])
            ls[features.columns.get_loc(j[k]) + 2] = abs(int(j[k+1]) - (int(j[k+2])*.99))
        
#     features.loc[i-1]=ls
    features.loc[i]=ls

features

,Education/Knowledge/Street Smarts,partner_education_score,difference_education_score,Financial Choices,partner_financial,difference_financial,Confidence/Self-Esteem,partner_confidence,difference_confidence,Religious/Spiritual Values,...,difference_culture,Self-Care/Personal Hygiene/Cleanliness,partner_care,difference_care,Honesty/Dependable/Reliable,partner_honesty,difference_honesty,Family Values,partner_family,difference_family
1,5.0,6.0,0.94,0.0,0.0,0.00,5.0,4.0,1.04,0.0,...,0.0,0.0,0.0,0.00,6.0,8.0,1.92,4.0,7.0,2.93
2,8.0,8.0,0.08,9.0,10.0,0.90,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.00,9.0,10.0,0.90,10.0,10.0,0.10
3,8.0,5.0,3.05,6.0,5.0,1.05,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00
4,8.0,8.0,0.08,8.0,8.0,0.08,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.00,9.0,9.0,0.09,8.0,9.0,0.91
5,7.0,8.0,0.92,0.0,0.0,0.00,0.0,0.0,0.00,0.0,...,0.0,8.0,8.0,0.08,0.0,0.0,0.00,6.0,7.0,0.93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,9.0,7.0,2.07,0.0,0.0,0.00,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.00,9.0,5.0,4.05,0.0,0.0,0.00
77,0.0,0.0,0.00,0.0,0.0,0.00,5.0,3.0,2.03,0.0,...,0.0,0.0,0.0,0.00,7.0,5.0,2.05,0.0,0.0,0.00
78,0.0,0.0,0.00,10.0,10.0,0.10,0.0,0.0,0.00,0.0,...,0.0,10.0,10.0,0.10,9.0,10.0,0.90,0.0,0.0,0.00
79,0.0,0.0,0.00,9.0,9.0,0.09,9.0,9.0,0.09,6.0,...,0.0,0.0,0.0,0.00,9.0,9.0,0.09,0.0,0.0,0.00


In [18]:
successful

,65,66,67,68,69,70,71,72,73,74,...,85,86,87,88,89,90,91,92,93,94
0,I consider myself emotionally intelligent,0,0,He/she was manipulative,We had great sexual chemistry throughout the r...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,I consider myself emotionally intelligent,I was jealous,0,0,We had great sexual chemistry throughout the r...,I was sexually/physically attracted in the beg...,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,I consider myself emotionally intelligent,0,0,0,0,0,0,I often wanted more attention/affection than m...,0,0,...,0,0,0,We had good communication,0,0,I was able to stay true to my identity,0,0,0
3,I consider myself emotionally intelligent,0,0,0,0,I was sexually/physically attracted in the beg...,0,0,0,0,...,0,We had on/off relationship/situationship,0,0,0,0,0,0,0,0
4,0,I was jealous,0,0,0,0,0,0,0,0,...,0,0,0,We had good communication,0,0,I was able to stay true to my identity,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,I consider myself emotionally intelligent,I was jealous,0,0,0,I was sexually/physically attracted in the beg...,0,I often wanted more attention/affection than m...,I felt I could do better,I felt I was not good enough for him/her,...,0,We had on/off relationship/situationship,0,0,0,0,0,0,We moved too fast,0
74,0,0,0,He/she was manipulative,We had great sexual chemistry throughout the r...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
75,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,I was able to stay true to my identity,0,0,0
76,I consider myself emotionally intelligent,0,0,0,0,0,0,0,0,0,...,0,0,0,We had good communication,0,0,I was able to stay true to my identity,0,0,0


(78, 107)

In [25]:
successful = lovedata.iloc[:,65:95]
successful = successful.fillna(0)
successful.reset_index(inplace=True, drop = True)
successful
total_list = []

for i, column in successful.iterrows():

        
    total = 0
    for j in range(len(column)):
        if column[j+65] != 0:
            successful.iloc[i,j] = 1
            total += 1
            
    total_list.append(total*10) #multiplied by ten to increase scale take out for true values
print(len(total_list)) 
print(successful.shape)
successful['Total'] = total_list

unsuccess_value = successful.loc[1:,['Total']]

#The Y values
unsuccess_value

78
(78, 30)


,Total
1,40
2,70
3,30
4,50
5,80
...,...
73,130
74,40
75,20
76,30


In [ ]:
conn = psycopg2.connect(user = config.user,
                              password = config.password,
                              host = config.hostname,
                              port = config.port,
                              database = config.database)

cur = conn.cursor()

# Print PostgreSQL version
cur.execute("SELECT * from profiles;")
record = cur.fetchall()

feat = pd.DataFrame(columns = ['Education/Knowledge/Street Smarts',
 'Financial Choices',
 'Confidence/Self-Esteem',
 'Religious/Spiritual Values',
 'Materialism/Superficiality',
 'Image/Fashion Sense/Body Modification',
 'Occupation/Work Ethic/Self-Discipline',
 'Household Care, Maintenance and Cleanliness',
 'Communication Style/Manners',
 'Artsy/Creative/Musical',
 'Charitable/Philanthropic',
 'Pursuing a Greater Purpose',
 'Social Status/Sociability',
 'Cultured/Well-traveled/"Woke"',
 'Self-Care/Personal Hygiene/Cleanliness',
 'Honesty/Dependable/Reliable',
 'Family Values'])

weights = [30,25,20,15,10]
j = 0
na = False
for x in record:
    #print(x[25:30])
    ls = [0 for x in range(len(feat.columns))]
    i = 0
    for k in range(25,39,3):
        if x[k] in feat.columns:
            ls[feat.columns.get_loc(x[k])] = weights[i]
#         elif "Other" in x[k]:
            na = True
            break
        i += 1
    if na:
        na = False
        continue
    feat.loc[j,:] = ls
    j += 1

In [27]:
X = features
y = unsuccess_value


In [30]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)
model = LinearRegression()

# Fitting our model with all of our features in X
model.fit(X, y)

score = model.score(X, y)
print(f"R2 Score: {score}")

model.fit(X_train, y_train)
training_score = model.score(X_train, y_train)
testing_score = model.score(X_test, y_test)
print(f"Training Score: {training_score}")
print(f"Testing Score: {testing_score}")

print("coefficient values")
print(model.coef_)
print("-----------------------------------")
print("intercept")
print(model.intercept_)
print("-----------------------------------")

R2 Score: 0.7023222256021262


Training Score: 0.8352035833422348
Testing Score: -222.6484314910116


coefficient values
[[-6.77593179e+00  6.00723888e+00  8.19150798e+00  1.75721861e+00
  -5.17308884e+00  8.75595007e+00 -1.34934467e+01  1.08694631e+01
   4.51907033e+00 -9.89667828e+00  9.06446487e+00 -2.90837208e+00
  -2.34923192e-13  7.31645855e-11  3.20987681e-11  7.52463000e+02
  -7.50925328e+02  7.63477380e+02 -5.90185770e+00  6.04162342e+00
   4.94211905e+00  3.99665902e+00  5.00286730e+00 -1.22050657e+01
  -2.89666969e+00  5.31933952e+00  2.66504332e+00 -3.87413114e+00
   3.50078896e+00 -7.65370908e+00  2.67159639e+00 -1.64971931e+00
   4.30481851e+00  1.55398787e+01 -1.39889128e+01 -1.60754820e+00
   8.40870501e-01 -6.85271715e-01  6.29132870e-01 -7.84439180e-12
   6.02540240e-12 -4.71800377e-12 -1.18881179e+01  1.19744400e+01
   8.34507973e+00  1.84758783e+00  1.38010518e-02  3.87815171e+00
   5.48732332e+00 -6.67742187e+00  4.16258749e-01]]
-----------------------------------
intercept
[30.28176567]
-----------------------------------


In [37]:
predictions = model.predict(X)
pd.DataFrame({"Predicted": list(predictions), "Actual": list(y.to_numpy()), "Error": list(predictions - y.to_numpy())})[["Predicted", "Actual", "Error"]]

,Predicted,Actual,Error
0,[49.38224917304035],[40],[9.38224917304035]
1,[78.5288227415684],[70],[8.528822741568405]
2,[32.460664683501136],[30],[2.460664683501136]
3,[0.9566712389438194],[50],[-49.04332876105618]
4,[75.0253909503899],[80],[-4.974609049610095]
...,...,...,...
72,[116.01777485481213],[130],[-13.982225145187869]
73,[52.37281599719708],[40],[12.37281599719708]
74,[20.844894672914336],[20],[0.8448946729143358]
75,[29.999999999994735],[30],[-5.265121671982342e-12]
